## 项目背景
> 找出10个成绩最为相似的学生，即各科成绩相差的绝对值最小的10个同学。比如:
   > - 两个学生的成绩为Student1(83,83,83,83,83,83,84);Student2(83,83,83,83,83,83,84)
   > - Distance(Student1，Student2)=|83-83|+|83-83|+|83-83|+|83-83|+|83-83|+|84-84|=0;
   > - 则判断这两个学生的成绩最为接近。

## 解题思路1：笛卡尔积+相似度计算

- 1.使用笛卡尔积的方式，得到每个学生和另外所有学生的关联行
> 每个学生都要与其他所有行进行交叉关联，比如有100个学生，那应该得到100*100 = 1W行结果

- 2.对于关联行，使用df.apply(function)的方法，计算两两相似度；
> 本步骤是为了计算每个学生和其他学生的 相似度

- 3.使用groupby + top n的方式，计算每个学生成绩最相近的10个学生

### 读取数据

In [1]:
import pandas as pd
df = pd.read_excel('data/成绩.xlsx', sheet_name='100_6', engine="openpyxl")
df.head()

,姓名,C语言程序设计,线性代数,大学英语,高等数学,管理学原理,大学计算机
0,学生1,86,92,71,89,89,85
1,学生2,85,95,75,95,82,83
2,学生3,78,84,77,94,87,81
3,学生4,73,84,82,95,87,72
4,学生5,84,94,83,97,77,80


In [2]:
df.shape

(100, 7)

### 笛卡尔积

In [3]:
df['one'] = 1
df.head()

,姓名,C语言程序设计,线性代数,大学英语,高等数学,管理学原理,大学计算机,one
0,学生1,86,92,71,89,89,85,1
1,学生2,85,95,75,95,82,83,1
2,学生3,78,84,77,94,87,81,1
3,学生4,73,84,82,95,87,72,1
4,学生5,84,94,83,97,77,80,1


In [4]:
df_merge = pd.merge(left=df, right=df, on='one')
df_merge.head()

,姓名_x,C语言程序设计_x,线性代数_x,大学英语_x,高等数学_x,管理学原理_x,大学计算机_x,one,姓名_y,C语言程序设计_y,线性代数_y,大学英语_y,高等数学_y,管理学原理_y,大学计算机_y
0,学生1,86,92,71,89,89,85,1,学生1,86,92,71,89,89,85
1,学生1,86,92,71,89,89,85,1,学生2,85,95,75,95,82,83
2,学生1,86,92,71,89,89,85,1,学生3,78,84,77,94,87,81
3,学生1,86,92,71,89,89,85,1,学生4,73,84,82,95,87,72
4,学生1,86,92,71,89,89,85,1,学生5,84,94,83,97,77,80


In [5]:
df_merge.shape

(10000, 15)

### 两两相似度计算

In [6]:
columns = df.columns.tolist()
columns.remove('姓名')
columns.remove('one')
columns

['C语言程序设计', '线性代数', '大学英语', '高等数学', '管理学原理', '大学计算机']

In [7]:
def sim_func(row):
    sim_value = 0.0
    for col in columns:
        sim_value += abs(int(row[col + '_x']) - int(row[col + '_y']))
    return sim_value

In [8]:
df_merge['sim'] = df_merge.apply(sim_func, axis=1)

In [9]:
df_merge.head()

,姓名_x,C语言程序设计_x,线性代数_x,大学英语_x,高等数学_x,管理学原理_x,大学计算机_x,one,姓名_y,C语言程序设计_y,线性代数_y,大学英语_y,高等数学_y,管理学原理_y,大学计算机_y,sim
0,学生1,86,92,71,89,89,85,1,学生1,86,92,71,89,89,85,0.0
1,学生1,86,92,71,89,89,85,1,学生2,85,95,75,95,82,83,23.0
2,学生1,86,92,71,89,89,85,1,学生3,78,84,77,94,87,81,33.0
3,学生1,86,92,71,89,89,85,1,学生4,73,84,82,95,87,72,53.0
4,学生1,86,92,71,89,89,85,1,学生5,84,94,83,97,77,80,41.0


In [10]:
# 把每个学生自己和自己关联的行去掉
df_merge = df_merge[df_merge['姓名_x'] != df_merge['姓名_y']]

In [11]:
df_merge.head()

,姓名_x,C语言程序设计_x,线性代数_x,大学英语_x,高等数学_x,管理学原理_x,大学计算机_x,one,姓名_y,C语言程序设计_y,线性代数_y,大学英语_y,高等数学_y,管理学原理_y,大学计算机_y,sim
1,学生1,86,92,71,89,89,85,1,学生2,85,95,75,95,82,83,23.0
2,学生1,86,92,71,89,89,85,1,学生3,78,84,77,94,87,81,33.0
3,学生1,86,92,71,89,89,85,1,学生4,73,84,82,95,87,72,53.0
4,学生1,86,92,71,89,89,85,1,学生5,84,94,83,97,77,80,41.0
5,学生1,86,92,71,89,89,85,1,学生6,85,85,66,98,83,72,41.0


### 计算每个学生成绩最相近的十个学生

In [12]:
def get_top_student(df_sub):
    df_sort = df_sub.sort_values(by='sim', ascending=True).head(10)
    names = ','.join(list(df_sort['姓名_y']))
    sims = ','.join(str(x) for x in df_sort['sim'].tolist())
    return pd.Series({'names': names, 'sims': sims})

In [13]:
df_result = df_merge.groupby(by='姓名_x').apply(get_top_student)
df_result

,names,sims
姓名_x,,
学生1,"学生8,学生2,学生3,学生11,学生21,学生5,学生6,学生20,学生10,学生19","18.0,23.0,33.0,38.0,40.0,41.0,41.0,43.0,44.0,45.0"
学生10,"学生26,学生2,学生6,学生18,学生28,学生5,学生30,学生12,学生34,学生13","19.0,29.0,29.0,38.0,38.0,39.0,39.0,39.0,40.0,41.0"
学生100,"学生31,学生36,学生38,学生60,学生68,学生53,学生42,学生66,学生37,学生77","36.0,37.0,38.0,42.0,43.0,43.0,43.0,43.0,44.0,44.0"
学生11,"学生20,学生23,学生3,学生39,学生9,学生1,学生8,学生40,学生24,学生2","33.0,35.0,37.0,37.0,37.0,38.0,38.0,39.0,39.0,41.0"
学生12,"学生21,学生13,学生6,学生24,学生4,学生18,学生28,学生33,学生5,学生3","25.0,26.0,28.0,32.0,32.0,33.0,33.0,34.0,34.0,34.0"
...,...,...
学生95,"学生93,学生94,学生96,学生97,学生89,学生92,学生91,学生84,学生88,学生80","43.0,54.0,55.0,56.0,61.0,63.0,64.0,71.0,72.0,73.0"
学生96,"学生93,学生94,学生95,学生92,学生91,学生87,学生89,学生88,学生97,学生90","44.0,53.0,55.0,56.0,57.0,64.0,64.0,65.0,69.0,71.0"
学生97,"学生81,学生83,学生89,学生93,学生94,学生91,学生72,学生80,学生92,学生95","36.0,37.0,41.0,45.0,46.0,52.0,53.0,53.0,55.0,56.0"


## 解题思路2：双重for循环+最小堆解决需求
堆是数据结构中最常见的一种数据结构，是一个完全二叉树。最小堆中每一个节点的值都小于等于其子树中每个节点的值。

具体的原理，学过数据结构的读者都懂，没有学过的也不用深究。

上面的代码中使用最小堆的目的是为了避免排序，采用堆化方式查找N个最小值，基于最小堆的查找，时间复杂度在O(logN)~O(N)之间，

远比排序的时间复杂度O(nlogN)快。

关于heapq的官方文档：https://docs./zh-cn/3/library/heapq.html?highlight=heapq#module-heapq

heapq的实现源码：https://github.com/python/cpython/blob/3.9/Lib/heapq.py

In [14]:
import numpy as np
import heapq

In [15]:
data = df.values
length = data.shape[0]
names = data[:, 0]
scores = data[:, 1:]

In [16]:
result = []
# 遍历取出每一行的数据
for i in range(length):
    # 分别取出当前遍历出来的姓名和成绩列表
    name, score = names[i], scores[i]
    # 用一个最小堆保存最接近的10学生
    min_similar_10 = []
    for j in range(length):
        if i == j:
            # 跳过对自己的比较
            continue
        # 被比较的学生姓名和成绩
        find_name, find_scores = names[j], scores[j]
        # 计算两个学生成绩的距离
        sim_value = np.abs(score-find_scores).sum()
        # 将被比较的学生和对应距离保存到最小堆中
        heapq.heappush(min_similar_10, (find_name, sim_value))
    # 取出10个距离最小的学生
    min_similar_10 = heapq.nsmallest(10, min_similar_10, key=lambda x: x[1])
    name_distance_dict = dict(min_similar_10)
    name_similars, distances = list(name_distance_dict.keys()), list(name_distance_dict.values())
    result.append((name, name_similars, distances))

In [17]:
result = pd.DataFrame(result, columns=["姓名", "分数最接近的10个学生", "距离"])
result

,姓名,分数最接近的10个学生,距离
0,学生1,"[学生8, 学生2, 学生3, 学生11, 学生21, 学生5, 学生6, 学生20, 学生...","[18, 23, 33, 38, 40, 41, 41, 43, 44, 45]"
1,学生2,"[学生8, 学生5, 学生1, 学生3, 学生10, 学生7, 学生6, 学生13, 学生1...","[19, 20, 23, 28, 29, 31, 34, 36, 36, 37]"
2,学生3,"[学生4, 学生15, 学生7, 学生13, 学生2, 学生18, 学生23, 学生17, ...","[20, 27, 27, 28, 28, 29, 30, 31, 31, 32]"
3,学生4,"[学生13, 学生3, 学生15, 学生29, 学生18, 学生17, 学生12, 学生24...","[20, 20, 21, 24, 25, 27, 32, 32, 33, 36]"
4,学生5,"[学生2, 学生7, 学生13, 学生8, 学生14, 学生12, 学生3, 学生24, 学...","[20, 21, 28, 31, 32, 34, 36, 38, 38, 39]"
...,...,...,...
95,学生96,"[学生93, 学生94, 学生95, 学生92, 学生91, 学生87, 学生89, 学生8...","[44, 53, 55, 56, 57, 64, 64, 65, 69, 71]"
96,学生97,"[学生81, 学生83, 学生89, 学生93, 学生94, 学生91, 学生72, 学生8...","[36, 37, 41, 45, 46, 52, 53, 53, 55, 56]"
97,学生98,"[学生42, 学生55, 学生69, 学生53, 学生58, 学生31, 学生41, 学生2...","[45, 47, 47, 51, 51, 52, 52, 53, 54, 54]"
98,学生99,"[学生100, 学生70, 学生49, 学生90, 学生51, 学生39, 学生76, 学生...","[67, 73, 75, 77, 78, 80, 82, 84, 85, 87]"


## 解题思路3：numpy向量化操作

利用numpy广播变量的特性一次性求出该学生与所有学生（含自己）的距离：

In [18]:
result = []
for i in range(length):
    name, score = names[i], scores[i]
    # 利用numpy广播变量的特性一次性求出该学生与所有学生（含自己）的距离
    score_diff = np.abs(scores-score).sum(axis=1)
    min_similar_index = np.argpartition(score_diff, 11)[:11].tolist()
    min_similar_index.remove(i)
    name_similars = names[min_similar_index]
    distances = score_diff[min_similar_index]
    result.append((name, name_similars, distances))

In [19]:
result = pd.DataFrame(result, columns=["姓名", "分数最接近的10个学生", "距离"])
result

,姓名,分数最接近的10个学生,距离
0,学生1,"[学生8, 学生2, 学生3, 学生11, 学生21, 学生5, 学生6, 学生20, 学生...","[18, 23, 33, 38, 40, 41, 41, 43, 44, 45]"
1,学生2,"[学生8, 学生5, 学生1, 学生3, 学生10, 学生7, 学生6, 学生13, 学生1...","[19, 20, 23, 28, 29, 31, 34, 36, 36, 37]"
2,学生3,"[学生4, 学生7, 学生15, 学生2, 学生13, 学生18, 学生23, 学生25, ...","[20, 27, 27, 28, 28, 29, 30, 31, 31, 32]"
3,学生4,"[学生3, 学生13, 学生15, 学生29, 学生18, 学生17, 学生24, 学生12...","[20, 20, 21, 24, 25, 27, 32, 32, 33, 36]"
4,学生5,"[学生2, 学生7, 学生13, 学生8, 学生14, 学生12, 学生3, 学生24, 学...","[20, 21, 28, 31, 32, 34, 36, 38, 38, 39]"
...,...,...,...
95,学生96,"[学生93, 学生94, 学生95, 学生92, 学生91, 学生87, 学生89, 学生8...","[44, 53, 55, 56, 57, 64, 64, 65, 69, 71]"
96,学生97,"[学生81, 学生83, 学生89, 学生93, 学生94, 学生91, 学生72, 学生8...","[36, 37, 41, 45, 46, 52, 53, 53, 55, 56]"
97,学生98,"[学生42, 学生55, 学生69, 学生53, 学生58, 学生31, 学生41, 学生2...","[45, 47, 47, 51, 51, 52, 52, 53, 54, 54]"
98,学生99,"[学生100, 学生70, 学生49, 学生90, 学生51, 学生39, 学生76, 学生...","[67, 73, 75, 77, 78, 80, 82, 84, 85, 87]"


## 解题思路4：KNN算法

In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [21]:
# 取出用于被KNN训练的数据
data = df.iloc[:, 1:].values
# y本身用于标注每条数据属于哪个类别，但我并不使用KNN的分类功能，所以统一全部标注为类别0
y = np.zeros(data.shape[0], dtype='int8')

knn = KNeighborsClassifier(n_neighbors=1, algorithm='ball_tree', p=1)
knn.fit(data, y)
distance, similar_points = knn.kneighbors(data, n_neighbors=11, return_distance=True)
distance = distance.astype("int", copy=False)
names = df['姓名']
result = []
for i, name in names.iteritems():
    name_similar_indexs = similar_points[i].tolist()
    self_index = name_similar_indexs.index(i)
    name_similar_indexs.pop(self_index)
    name_similars = names[name_similar_indexs].tolist()
    distances = distance[i].tolist()
    distances.pop(self_index)
    result.append((name, name_similars, distances))

In [22]:
result = pd.DataFrame(result, columns=["姓名", "分数最接近的10个学生", "距离"])
result

,姓名,分数最接近的10个学生,距离
0,学生1,"[学生8, 学生2, 学生3, 学生11, 学生21, 学生6, 学生5, 学生20, 学生...","[18, 23, 33, 38, 40, 41, 41, 43, 44, 45]"
1,学生2,"[学生8, 学生5, 学生1, 学生3, 学生10, 学生7, 学生6, 学生13, 学生1...","[19, 20, 23, 28, 29, 31, 34, 36, 36, 37]"
2,学生3,"[学生4, 学生15, 学生7, 学生2, 学生13, 学生18, 学生23, 学生25, ...","[20, 27, 27, 28, 28, 29, 30, 31, 31, 32]"
3,学生4,"[学生3, 学生13, 学生15, 学生29, 学生18, 学生17, 学生12, 学生24...","[20, 20, 21, 24, 25, 27, 32, 32, 33, 36]"
4,学生5,"[学生2, 学生7, 学生13, 学生8, 学生14, 学生12, 学生3, 学生24, 学...","[20, 21, 28, 31, 32, 34, 36, 38, 38, 39]"
...,...,...,...
95,学生96,"[学生93, 学生94, 学生95, 学生92, 学生91, 学生87, 学生89, 学生8...","[44, 53, 55, 56, 57, 64, 64, 65, 69, 71]"
96,学生97,"[学生81, 学生83, 学生89, 学生93, 学生94, 学生91, 学生72, 学生8...","[36, 37, 41, 45, 46, 52, 53, 53, 55, 56]"
97,学生98,"[学生42, 学生55, 学生69, 学生58, 学生53, 学生31, 学生41, 学生2...","[45, 47, 47, 51, 51, 52, 52, 53, 54, 54]"
98,学生99,"[学生100, 学生70, 学生49, 学生90, 学生51, 学生39, 学生76, 学生...","[67, 73, 75, 77, 78, 80, 82, 84, 85, 87]"
